In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib


import os,sys
from importlib import reload
import copy
from collections import namedtuple

sys.path.append("../tracker")

In [193]:
import kalmanfilter as KF
import utilities as Util
import datatypes
import vertexfinder as VF

reload(VF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [204]:
tracks=joblib.load("tracks_example.joblib")
# tracks

In [207]:
vf = VF.VertexFitter(debug=True)
vf.parameters["cut_vertex_SeedDist"]=300
vf.parameters["cut_vertex_SeedChi2"]=80
vf.parameters["cut_vertex_TrackAddDist"]=300
vf.parameters["cut_vertex_TrackAddChi2"]=20
vf.parameters["cut_vertex_VertexChi2Reduced"]=5
vertices = vf.run(tracks)
# %timeit -n 2 vf.run(tracks)

VertexSeed(x0=694.1101749650073, y0=9574.74801130732, z0=9003.587337266003, t0=3.300594043692026, cov=0, chi2=73.84963520498309, dist=126.8861882757568, Ntracks=5, trackind1=0, trackind2=7, score=2082.411942800803)
VertexSeed(x0=703.1974988454831, y0=9547.382079407675, z0=8955.339203987269, t0=1.527055869364827, cov=0, chi2=29.74569690303885, dist=114.73431506392902, Ntracks=5, trackind1=5, trackind2=7, score=1942.8917832957382)
VertexSeed(x0=702.0285664560897, y0=9627.416780009311, z0=9145.00169169731, t0=4.700390869811811, cov=0, chi2=34.374336753019854, dist=98.69811355577785, Ntracks=5, trackind1=1, trackind2=3, score=1926.7314749150885)
VertexSeed(x0=699.2576518275425, y0=9589.027249315197, z0=9106.6188861388, t0=6.213863439326124, cov=0, chi2=3.420680257287523, dist=112.3942433722316, Ntracks=5, trackind1=3, trackind2=7, score=1870.6459849873136)
VertexSeed(x0=759.3348217852675, y0=9757.613766606144, z0=9183.875447490878, t0=10.518501727158053, cov=0, chi2=3.0441696104370406, dis

In [182]:
0.01*(7362-7000-1950), 0.01*(-232.8637214423652), 0.01*(9785.048555922185-8547) 

(-15.88, -2.3286372144236522, 12.380485559221853)

In [69]:
print(len(vertices))
print(len(vertices[0].tracks))
print(vertices[0])

0


IndexError: list index out of range

In [ ]:
def root_hits_extractor_tracks(Tree, entry):
    Tree.GetEntry(entry)
    c2list = lambda x: np.array([x.at(i) for i in range(x.size())])

    Digi_layer = c2list(Tree.Digi_layer_id)
    # Only select hits in tracking layer
    mask = (Digi_layer>1) & (Digi_layer<=5) 

    Digi_x = c2list(Tree.Digi_x)[mask]
    Digi_y = c2list(Tree.Digi_y)[mask]
    Digi_z = c2list(Tree.Digi_z)[mask]
    Digi_t = c2list(Tree.Digi_time)[mask]
    Digi_track_id = c2list(Tree.Digi_track_id)[mask]
    Digi_layer = Digi_layer[mask]
    Hits = Util.hit.make_hits(Digi_x, Digi_y, Digi_z, Digi_t, Digi_layer)

    track_ids = np.unique(Digi_track_id)
    track_hits  = {}
    for trackid in track_ids:
        mask = Digi_track_id==trackid
        hit_group = []
        for i in np.flatnonzero(mask):
            hit_group.append(Hits[i])
        track_hits[int(trackid)] = hit_group

    return track_hits